In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 초음파 광물 예측하기

#### 1) 데이터 확인과 실행

In [2]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
# seed 값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [4]:
# 데이터 입력
df = pd.read_csv("/content/gdrive/My Drive/deepLearning/data/sonar (1).csv", header=None)

In [5]:
dataset = df.values
X = dataset[:,0:60].astype(float)
Y_obj = dataset[:,60]

# 문자열 변환
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

In [6]:
# 모델 설정
model = Sequential()
model.add(Dense(24,  input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [7]:
# 모델 컴파일
model.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['accuracy'])

In [8]:
# 모델 실행
model.fit(X, Y, epochs=200, batch_size=5)

Epoch 1/200
42/42 [==============================] - 0s 1ms/step - loss: 0.2468 - accuracy: 0.5096
Epoch 2/200
42/42 [==============================] - 0s 1ms/step - loss: 0.2269 - accuracy: 0.6971
Epoch 3/200
42/42 [==============================] - 0s 1ms/step - loss: 0.2153 - accuracy: 0.7115
Epoch 4/200
42/42 [==============================] - 0s 1ms/step - loss: 0.2048 - accuracy: 0.7115
Epoch 5/200
42/42 [==============================] - 0s 1ms/step - loss: 0.1969 - accuracy: 0.7260
Epoch 6/200
42/42 [==============================] - 0s 1ms/step - loss: 0.1871 - accuracy: 0.7692
Epoch 7/200
42/42 [==============================] - 0s 1ms/step - loss: 0.1768 - accuracy: 0.7788
Epoch 8/200
42/42 [==============================] - 0s 993us/step - loss: 0.1715 - accuracy: 0.7837
Epoch 9/200
42/42 [==============================] - 0s 1ms/step - loss: 0.1620 - accuracy: 0.8125
Epoch 10/200
42/42 [==============================] - 0s 990us/step - loss: 0.1619 - accuracy: 0.7981
Epoch

In [9]:
# 결과 출력
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

7/7 [==============================] - 0s 1ms/step - loss: 0.0034 - accuracy: 1.0000

 Accuracy: 1.0000


#### 2) 학습셋과 테스트셋, 모델 저장과 재사용

In [26]:
from keras.models import Sequential, load_model
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import tensorflow as tf

In [11]:
# seed 값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [12]:
# 데이터 입력
df = pd.read_csv("/content/gdrive/My Drive/deepLearning/data/sonar (1).csv", header=None)

In [19]:
dataset = df.values
X = dataset[:,0:60].astype(float)
Y_obj = dataset[:,60]

In [20]:
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

In [21]:
# 학습 셋과 테스트 셋의 구분
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, 
                                                    random_state=seed)

In [22]:
model = Sequential()
model.add(Dense(24,  input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [23]:
model.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['accuracy'])

In [24]:
model.fit(X_train, Y_train, epochs=130, batch_size=5)

Epoch 1/130
29/29 [==============================] - 0s 1ms/step - loss: 0.2506 - accuracy: 0.4690
Epoch 2/130
29/29 [==============================] - 0s 1ms/step - loss: 0.2418 - accuracy: 0.5793
Epoch 3/130
29/29 [==============================] - 0s 1ms/step - loss: 0.2364 - accuracy: 0.6207
Epoch 4/130
29/29 [==============================] - 0s 1ms/step - loss: 0.2301 - accuracy: 0.6621
Epoch 5/130
29/29 [==============================] - 0s 1ms/step - loss: 0.2242 - accuracy: 0.6828
Epoch 6/130
29/29 [==============================] - 0s 1ms/step - loss: 0.2172 - accuracy: 0.6966
Epoch 7/130
29/29 [==============================] - 0s 1ms/step - loss: 0.2104 - accuracy: 0.7310
Epoch 8/130
29/29 [==============================] - 0s 1ms/step - loss: 0.2086 - accuracy: 0.6897
Epoch 9/130
29/29 [==============================] - 0s 1ms/step - loss: 0.1972 - accuracy: 0.7103
Epoch 10/130
29/29 [==============================] - 0s 1ms/step - loss: 0.1917 - accuracy: 0.7517
Epoch 11/

In [27]:
model.save('/content/gdrive/My Drive/deepLearning/model/my_model.h5')

In [28]:
del model
model=load_model('/content/gdrive/My Drive/deepLearning/model/my_model.h5')

In [29]:
# 테스트셋에 모델 적용
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

2/2 [==============================] - 0s 3ms/step - loss: 0.1369 - accuracy: 0.8254

 Test Accuracy: 0.8254


#### 3) k겹 교차 검증

In [30]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import numpy as np
import pandas as pd
import tensorflow as tf

In [32]:
# seed 값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [33]:
# 데이터 입력
df = pd.read_csv("/content/gdrive/My Drive/deepLearning/data/sonar (1).csv", header=None)

In [34]:
dataset = df.values
X = dataset[:,0:60].astype(float)
Y_obj = dataset[:,60]

In [35]:
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

In [36]:
# 10개의 파일로 쪼갬
n_fold = 10
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [37]:
# 빈 accuracy 배열
accuracy = []

# 모델의 설정, 컴파일, 실행
for train, test in skf.split(X, Y):
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    model.fit(X[train], Y[train], epochs=100, batch_size=5)
    
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)

Epoch 1/100
38/38 [==============================] - 0s 1ms/step - loss: 0.2426 - accuracy: 0.5455
Epoch 2/100
38/38 [==============================] - 0s 953us/step - loss: 0.2260 - accuracy: 0.6578
Epoch 3/100
38/38 [==============================] - 0s 990us/step - loss: 0.2156 - accuracy: 0.7326
Epoch 4/100
38/38 [==============================] - 0s 1ms/step - loss: 0.2057 - accuracy: 0.7487
Epoch 5/100
38/38 [==============================] - 0s 1ms/step - loss: 0.1946 - accuracy: 0.7219
Epoch 6/100
38/38 [==============================] - 0s 1ms/step - loss: 0.1843 - accuracy: 0.7540
Epoch 7/100
38/38 [==============================] - 0s 1ms/step - loss: 0.1730 - accuracy: 0.7861
Epoch 8/100
38/38 [==============================] - 0s 1ms/step - loss: 0.1631 - accuracy: 0.8075
Epoch 9/100
38/38 [==============================] - 0s 1ms/step - loss: 0.1559 - accuracy: 0.8075
Epoch 10/100
38/38 [==============================] - 0s 975us/step - loss: 0.1504 - accuracy: 0.8075
Epo

In [38]:
# 결과 출력
print("\n %.f fold accuracy:" % n_fold, accuracy)


 10 fold accuracy: ['0.7143', '0.8095', '0.7619', '0.9048', '0.8571', '0.7619', '0.8571', '0.7619', '0.9500', '0.8500']
